# Japanese fake news classification

We're going to attempt to classify this dataset. This is a dataset featuring Japanese news articles, from which a part
real, some are half fake, and some are entirely fake.

0: Original article
1: Partially fake
2: Completely fake

Source: [Japanese fakenews dataset](https://www.kaggle.com/tanreinama/japanese-fakenews-dataset)

## Inspection

In [1]:
import re

import numpy as np
import pandas as pd
import requests
from sudachipy import tokenizer, dictionary
from gensim.models import KeyedVectors
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense, Embedding, Flatten, Bidirectional, LSTM
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import tensorflow as tf

physical_devices = tf.config.list_physical_devices('GPU')
for i in physical_devices:
    tf.config.experimental.set_memory_growth(i, enable=True)

df = pd.read_csv("fakenews.csv")
df.describe().transpose()

a:\software\coding projects\aai\japanesetextclassification\venv\lib\site-packages\gensim\similarities\__init__.py:15: UserWarning: The gensim.similarities.levenshtein submodule is disabled, because the optional Levenshtein package <https://pypi.org/project/python-Levenshtein/> is unavailable. Install Levenhstein (e.g. `pip install python-Levenshtein`) to suppress this warning.
  warnings.warn(msg)


,count,mean,std,min,25%,50%,75%,max
isfake,13040.0,1.075613,0.796950,0.0,0.0,1.0,2.00,2.0
nchar_real,13040.0,261.029371,288.257753,0.0,0.0,215.0,407.00,4447.0
nchar_fake,13040.0,328.273160,354.673240,0.0,0.0,255.0,489.25,2582.0


In [2]:
print(f'Rows/Columns: {df.shape}')
print(f"Class distribution is: \n{df['isfake'].value_counts()}\n")
print(df.isnull().sum())

Rows/Columns: (13040, 5)
Class distribution is: 
1    4684
2    4671
0    3685
Name: isfake, dtype: int64

id            0
context       0
isfake        0
nchar_real    0
nchar_fake    0
dtype: int64


From what we can see the dataset does not contain null values, and it's class distribution is mostly even.

Although mostly even, not perfectly even, so while my first thought was accuracy as a metric, i've decided to use the
ROC curve instead.

In [3]:
print(df['context'].head())

0    朝日新聞など各社の報道によれば、宅配便最大手「ヤマト運輸」が日本郵政公社を相手取り、大手コン...
1    11月5日の各社報道によると、諫早湾干拓事業は諫早海人（諫早湾の「海」）に囲まれる大洋に位置...
2    産経新聞、中日新聞によると、2004年から2005年まで、この大会による3年おきの開催を、2...
3    開催地のリオデジャネイロ市に対して、大会期間中のリオデジャネイロオリンピックに関する公式発表...
4    毎日新聞・時事通信によると、2006年2月13日には、グッドウィル・グッゲンハイム・アン・ハ...
Name: context, dtype: object


## Preprocessing

We can mostly follow the standard NLP cleaning methods, albeit with a catch, it's a character based language, so the
traditional packages such as NLTK won't work. That's why we need to find substitutes for the cleaning methods. Let's go
over them:

I've decided to go with [SudachiPy](https://pypi.org/project/SudachiPy/) for tokenization and stemming
[stopwords-ja](https://github.com/stopwords-iso/stopwords-ja/blob/master/stopwords-ja.json) (link to json file) for stopword removal.

I decided to not use lemmatization, since there are a lot of words that might _seem_ similar, but are vastly different
taking into account their context.

### Removing features

Everything besides the actual text and the label class is just noise, so we'll take those out.

In [4]:
labels = df.pop('isfake')
data = df.pop('context')

### Tokenization

We're going to tokenize our sentences. This we'll do with Sudachi, a morphological analyzer. The short version of
what that means is that we're splitting our sentences up into pieces, aka tokens. We need a specialized analyzer for
this task, because you can't just randomly split it up by character, because then you'd lose context. After tokenization
is complete we can do the rest of the preprocessing steps, which we usually apply individually per character.

### Special characters

We need to also remove some special characters that will be of no use to the classifier, think of stuff such as commas
and question marks, etc. We do this via a regex filter. I spent some time looking for the exactly right one online,
since I wanted to keep all japanese characters, but remove junk such as symbols. Speaking of symbols for some reason
the Japanese have their own version of them for some reason (。、？  notice how they're different?), which is pretty
tricky. While you'd want to remove irrelevant characters, you also don't want to remove all symbols, since some are
very important to the language (e.g. ー, which lengthens vowels.).

The conclusion here is that you'll have to build your own japanese regex filter to suit your exact needs. Definitely
check out [this gist](https://gist.github.com/terrancesnyder/1345094) for that.

Check out the pre-processing function below for the specific regex i used.

### Stemming

Stemming implies reducing the form of a word to its stem. E.g. 食べている -->　食べる, した　--> する. This is useful to
reduce the amount of variants of a word which mostly mean the exact same thing (from the perspective of the model)

### Stopwords

Stopword removal implies the removal of certain words which appear in an excessive frequency in the language. Common
examples for English are "the", "and" and so forth. These are useful to remove because they lose meaning viewed
individually.

In [5]:
def sample(data, limit=3):
    """
    Quick util method to display samples of the sentences
    :param data: collection
    :param limit: how much samples to show
    """
    for x in range(0, limit, 1):
        print(f"{data[x]}\n")


sample(data)

朝日新聞など各社の報道によれば、宅配便最大手「ヤマト運輸」が日本郵政公社を相手取り、大手コンビニエンスストア「ローソン」でのサービス提供の差し止めなどを求めていた訴訟で、2006年1月19日、東京地方裁判所でヤマト運輸の請求を棄却する判決が下された。2004年のローソンでの郵便小包サービス「ゆうパック」の受付業務開始に際し、ヤマト運輸は「独占禁止法に違反する不当な廉売」として、日本郵政公社を相手取り、サービス提供の差し止めなどを求めていた。朝日新聞によれば、提訴の内容は、2004年11月のローソンでの「ゆうパック」の受付サービス提供の開始に関連し、租税などの優遇措置を受けている日本郵政公社が、配送料金（運賃）などの有利な取引条件でローソンで「ゆうパック」を開始させたのは、独占禁止法の不当廉売に当たり、ヤマト運輸の利益を侵害されるとして、「ゆうパック」サービス提供の差し止めなどを求めていたもの。朝日新聞によれば、判決内容はヤマト運輸の主張を全面的に否定しており、今後の「ゆうパック」サービスの拡大に弾みがつくものと考えられる。日本郵政公社は、公正妥当な判決とのコメントを出した。一方、ヤマト運輸は、高等裁判所への控訴など、今後の対応については検討するとアナウンスしている。

11月5日の各社報道によると、諫早湾干拓事業は諫早海人（諫早湾の「海」）に囲まれる大洋に位置することから、人身売買により、環境問題に加え、環境保護にも関心が向けられた。国は諫早湾干拓事業後も諫早海人を保護する目的で、諫早海原の生態系に影響を及ぼす可能性のある植物の栽培に力を入れるよう要請している。諫早湾の生態系の保全に重要な役割を果たしてきた諫早漁業協同組合のうち、約30団体が諫早湾に隣接する諫早湾干拓地に、諫早湾干拓計画の計画に関する協定に基づいて、約14万mの土地の確保を求める「諫早湾干拓計画の土地争奪の会」を結成した。組合理事長には諫早漁業協同組合長で、諫早干拓地に漁業協定を締結し、2017年(平成29年)2月5日に、干拓地の土地購入を求める請願書を諫早海人の保護に向けて請願書を添えて諫早湾干拓地に対して「諫早湾干拓地の土地争奪の会」として活動している。

産経新聞、中日新聞によると、2004年から2005年まで、この大会による3年おきの開催を、2006年から2年連続で実施したことがある。また、

In [6]:
def pre_process_text(text, stopwords, tokenizer_obj):
    """
    Cleans the text of unnecessary features
    :param text: sentence to clean
    :param stopwords: list of very commonly used words
    :param tokenizer_obj: SudachiPy tokenizer object
    :return: cleaned string
    """
    # One by one: (kanji), (hiragana and katakana), (western alphabet and numbers), (western alphabet and numbers
    # except it's the off-looking japanese version), and unicode flag
    pattern = re.compile(r'([一-龯]+)|([ぁ-んァ-ン]+)|([a-zA-Z0-9]+)|([ａ-ｚＡ-Ｚ０-９]+)|[ー+]', re.UNICODE)
    text = ''.join([x.group() for x in re.finditer(pattern, text.lower().strip())])

    # Tokenizes, converts to dictionary form, and doesn't add it if it's in the stopword list
    lst_text = [m.dictionary_form() for m in tokenizer_obj.tokenize(text, mode) if m not in stopwords]

    # Rejoin tokenized string
    text = " ".join(lst_text)
    return text


tokenizer_obj = dictionary.Dictionary().create()
mode = tokenizer.Tokenizer.SplitMode.C
lst_stopwords = requests.get(
    'https://raw.githubusercontent.com/stopwords-iso/stopwords-ja/master/stopwords-ja.json').json()

print("Starting cleaning phase, this may take a few minutes...")
data = [pre_process_text(i, lst_stopwords, tokenizer_obj) for i in data]
sample(data, 5)

Starting cleaning phase, this may take a few minutes...
朝日新聞 など 各社 の 報道 に よる ば 宅配便 最大手 ヤマト 運輸 が 日本郵政 公社 を 相手 取る 大手 コンビニエンスストアローソン で の サービス 提供 の 差し止め など を 求める て いる た 訴訟 で 2006 年 1 月 19 日 東京 地方裁判所 で ヤマト 運輸 の 請求 を 棄却 する 判決 が 下す れる た 2004 年 の ローソン で の 郵便 小包 サービス ゆう パック の 受付 業務 開始 に 際する ヤマト 運輸 は 独占禁止法 に 違反 する 不当 だ 廉売 と する て 日本郵政 公社 を 相手 取る サービス 提供 の 差し止め など を 求める て いる た 朝日新聞 に よる ば 提訴 の 内容 は 2004 年 11 月 の ローソン で の ゆう パック の 受付 サービス 提供 の 開始 に 関連 する 租税 など の 優遇措置 を 受ける て いる 日本郵政 公社 が 配送 料金 運賃 など の 有利 だ 取引 条件 で ローソン で ゆう パック を 開始 する せる た の は 独占禁止法 の 不当廉売 に 当たる ヤマト 運輸 の 利益 を 侵害 する れる と する て ゆう パック サービス 提供 の 差し止め など を 求める て いる た もの 朝日新聞 に よる ば 判決 内容 は ヤマト 運輸 の 主張 を 全面的 だ 否定 する て おる 今後 の ゆう パック サービス の 拡大 に 弾み が つく もの と 考える られる 日本郵政 公社 は 公正 妥当 だ 判決 と の コメント を 出す た 一方 ヤマト 運輸 は 高等裁判所 へ の 控訴 など 今後 の 対応 に つく て は 検討 する と アナウンス する て いる

11 月 5 日 の 各社 報道 に よる と 諫早湾 干拓 事業 は 諫早 海人 諫早湾 の 海 に 囲む れる 大洋 に 位置 する こと から 人身 売買 に よる 環境 問題 に 加える 環境 保護 に も 関心 が 向ける られる た 国 は 諫早湾 干拓 事業 後 も 諫早 海人 を 保護 する 目的 で 諫早 海原 の 生態系

## NLP technique

Now we need to choose a technique to actually process this text, because after all, our neural network only accepts
numbers, not japanese characters. Some examples of the current most popular techniques are Bag of Words, TF_IDF Scheme,
BERT, word2vec.

I've decided to use word2vec here, which maps words and/or phrases to vectors. word2vec comes with 2 options, Skip Gram
and Continuous Bag of Words, which are essentially mirrored versions of each other. CBOW is trained to predict a single
word from multiple context words, SG is trained to predict multiple words from a single context word.

I used [this pre-trained skip-gram embedding](https://github.com/singletongue/WikiEntVec/releases) trained on wikipedia articles,
which i got from [this comparison of Japanese embeddings](https://blog.hoxo-m.com/entry/2020/02/20/090000) since i
felt like it fit with the vocabulary of our news articles.

To load our embeddings i used Gensim, since it's the easiest to use.

In [ ]:
print("Loading word embedding into Gensim, this may take a while...")
w2vec_model = KeyedVectors.load_word2vec_format("jawiki.all_vectors.300d.txt", binary=False)

Loading word embedding into Gensim, this may take a while...


In [ ]:
# Example of vocabulary in the w2vec model
vocabulary = w2vec_model.index_to_key
print(vocabulary[10:20], "\n")

# Example of how to access vector of a word
print(w2vec_model['から'])


In [ ]:
weights = w2vec_model.vectors
vocabulary_length = weights.shape[0]
print(f"Imported word2vec model has {vocabulary_length} entries")

embedding_layer = Embedding(
    input_dim=weights.shape[0],
    output_dim=weights.shape[1],
    weights=[weights],
    trainable=False)

In [ ]:
t = Tokenizer()
t.fit_on_texts(data)
vocab_size = len(t.word_index) + 1

encoded_docs = t.texts_to_sequences(data)
print(encoded_docs[0])

padded_docs = pad_sequences(encoded_docs, padding='post')

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(padded_docs, labels, train_size=0.85, stratify=labels, shuffle=True)
X_train = np.asarray(X_train)
X_test = np.asarray(X_test)
y_train = np.asarray(y_train)
y_test = np.asarray(y_test)

In [ ]:
model = Sequential()
model.add(embedding_layer)
model.add(LSTM(1000))
model.add(Dense(1, activation='sigmoid'))


model.compile(optimizer='adam',
              loss='categorical_crossentropy')

In [ ]:
model.fit(X_train, y_train, epochs=100, batch_size=128, validation_split=0.1)

In [ ]:
nn_pred = model.predict(X_test)
nn_pred =  np.where(nn_pred > 0.5, 1, 0 )
ac = accuracy_score(y_test, nn_pred)

## Conclusion

Package wise, we still have a long way to go. The Japanese equivalents of the NLP packages are more underveloped,
less reliable, and generally slower too. This can be mostly attributed to the language being difficult, but it's
something you have to keep in mind while working with Japanese text. Maybe in a few years the tools will be at a good
level.


## References

[Preprocessing Methods and Tools in Modelling Japanese for Text Classification](https://www.researchgate.net/publication/335337209_Preprocessing_Methods_and_Tools_in_Modelling_Japanese_for_Text_Classification)
Paper detailing tools for Japanese NLP analysis. A few years old, but still relevant as of current date.

[nlp-recipes-ja](https://github.com/upura/nlp-recipes-ja)
Github repository containing a ton of samples for Japanese text analysis in Python.

[Migrating from Gensim 3.0 to 4.0](https://github.com/RaRe-Technologies/gensim/wiki/Migrating-from-Gensim-3.x-to-4)
Gensim recently upgraded to 4.0 release, which contains code-breaking API changes, and most recent material on gensim is
still using older versions.

[Gensim Keras embedding layer example](https://github.com/RaRe-Technologies/gensim/wiki/Using-Gensim-Embeddings-with-Keras-and-Tensorflow)
How to create a Keras embedding layer with a Gensim model

[Using pretrained gensim Word2vec embedding in keras](https://stackoverflow.com/q/60082554/7174982)
Helpful answer on how to encode the text sequences